In [1]:
import requests

url = "https://realtor.p.rapidapi.com/properties/v2/list-for-sale"

querystring = {"sort":"relevance","prop_type":"single_family","city":"San Diego","limit":"5000","offset":"0","state_code":"CA"}

headers = {
    'x-rapidapi-host': "realtor.p.rapidapi.com",
    'x-rapidapi-key': "ccd80db736mshcfc9234f86f9d33p12faf8jsn0eb2e033ed0c"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

response_json=response.json()

print(response_json.keys())

dict_keys(['meta', 'properties'])


In [0]:
properties = response_json['properties']

In [5]:
import pandas as pd
CITY_NAME_DF = pd.DataFrame(properties)
CITY_NAME_DF.head()

,property_id,listing_id,rdc_web_url,prop_type,address,branding,prop_status,price,baths_full,baths,beds,building_size,agents,office,last_update,client_display_flags,lead_forms,photo_count,thumbnail,page_no,rank,list_tracking,lot_size,mls,baths_half,virtual_tour,open_houses
0,M2278071853,2916019873,https://www.realtor.com/realestateandhomes-det...,single_family,"{'city': 'San Diego', 'line': '3910 Mount Blac...",{'listing_office': {'list_item': {'name': 'Big...,for_sale,829000,2,2,3,"{'size': 1352, 'units': 'sqft'}","[{'primary': True, 'advertiser_id': '1238460',...","{'id': 'b4c3e806585c74b3693a1a2c96e145a4', 'na...",2020-05-08T03:13:34Z,"{'presentation_status': 'for_sale', 'is_showca...","{'form': {'name': {'required': True, 'minimum_...",22,https://ap.rdcpix.com/775832d0e7b7488193f6b912...,1,1,type|property|data|prop_id|2278071853|list_id|...,"{'size': 6500, 'units': 'sqft'}","{'name': 'SanDiegoCountySPN', 'id': '200020739...",NaN,NaN,NaN
1,M2739002954,2916018041,https://www.realtor.com/realestateandhomes-det...,single_family,"{'city': 'San Diego', 'line': '8546 Boothbay P...",{'listing_office': {'list_item': {'name': 'Act...,for_sale,699000,2,2,3,"{'size': 1614, 'units': 'sqft'}","[{'primary': True, 'advertiser_id': '3886778',...","{'id': '755e3f62f485c48f26b29dbdde728971', 'na...",2020-05-08T03:07:27Z,"{'presentation_status': 'for_sale', 'is_showca...","{'form': {'name': {'required': True, 'minimum_...",25,https://ap.rdcpix.com/98da26dfbd8803f4f7eea9e3...,1,2,type|property|data|prop_id|2739002954|list_id|...,"{'size': 5718, 'units': 'sqft'}","{'name': 'SanDiegoCountySPN', 'id': '200020731...",NaN,NaN,NaN
2,M1281042411,2916017365,https://www.realtor.com/realestateandhomes-det...,single_family,"{'city': 'La Jolla', 'line': '1321 Virginia Wa...",{'listing_office': {'list_item': {'name': 'Kle...,for_sale,2395000,2,3,4,"{'size': 2586, 'units': 'sqft'}","[{'primary': True, 'advertiser_id': '1091393',...","{'id': '816848227acac88917b4c45f526c301a', 'na...",2020-05-08T01:24:19Z,"{'presentation_status': 'for_sale', 'is_showca...","{'form': {'name': {'required': True, 'minimum_...",25,https://ap.rdcpix.com/379dc82fe2cc4c820d7dba3b...,1,3,type|property|data|prop_id|1281042411|list_id|...,"{'size': 7700, 'units': 'sqft'}","{'name': 'SanDiegoCountySPN', 'id': '200020716...",1.0,NaN,NaN
3,M1811103700,2916017005,https://www.realtor.com/realestateandhomes-det...,single_family,"{'city': 'San Diego', 'line': '4127 Seri St', ...",{'listing_office': {'list_item': {'name': 'eXp...,for_sale,699999,2,2,3,"{'size': 1374, 'units': 'sqft'}","[{'primary': True, 'advertiser_id': '2026956',...","{'id': '58a0a4224e526cedc7acfefd51b56d90', 'na...",2020-05-08T01:04:22Z,"{'presentation_status': 'for_sale', 'is_showca...","{'form': {'name': {'required': True, 'minimum_...",25,https://ap.rdcpix.com/e6f873865af4796c8b2972b5...,1,4,type|property|data|prop_id|1811103700|list_id|...,"{'size': 6300, 'units': 'sqft'}","{'name': 'SanDiegoCountySPN', 'id': '200020715...",NaN,NaN,NaN
4,M1479304073,2916016538,https://www.realtor.com/realestateandhomes-det...,single_family,"{'city': 'San Diego', 'line': '15066 Almond Or...",{'listing_office': {'list_item': {'name': 'Rea...,for_sale,1450000,5,6,5,"{'size': 5390, 'units': 'sqft'}","[{'primary': True, 'advertiser_id': '2794409',...","{'id': '92099090911deb7430fff495e4157806', 'na...",2020-05-08T01:11:11Z,"{'presentation_status': 'for_sale', 'is_showca...","{'form': {'name': {'required': True, 'minimum_...",25,https://ap.rdcpix.com/ca732a98cce84dd20e382a0d...,1,5,type|property|data|prop_id|1479304073|list_id|...,"{'size': 23958, 'units': 'sqft'}","{'name': 'SanDiegoCountySPN', 'id': '200020710...",1.0,{'href': 'https://my.matterport.com/show/?m=1k...,NaN


In [7]:
sandiego_listings = CITY_NAME_DF[['property_id', 'prop_type', 'address', 'price', 'baths_half', 'baths_full', 'beds', 'building_size', 'lot_size']]
sandiego_listings["baths_half"] = sandiego_listings["baths_half"].fillna(0)
sandiego_listings

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,property_id,prop_type,address,price,baths_half,baths_full,beds,building_size,lot_size
0,M2278071853,single_family,"{'city': 'San Diego', 'line': '3910 Mount Blac...",829000,0.0,2,3,"{'size': 1352, 'units': 'sqft'}","{'size': 6500, 'units': 'sqft'}"
1,M2739002954,single_family,"{'city': 'San Diego', 'line': '8546 Boothbay P...",699000,0.0,2,3,"{'size': 1614, 'units': 'sqft'}","{'size': 5718, 'units': 'sqft'}"
2,M1281042411,single_family,"{'city': 'La Jolla', 'line': '1321 Virginia Wa...",2395000,1.0,2,4,"{'size': 2586, 'units': 'sqft'}","{'size': 7700, 'units': 'sqft'}"
3,M1811103700,single_family,"{'city': 'San Diego', 'line': '4127 Seri St', ...",699999,0.0,2,3,"{'size': 1374, 'units': 'sqft'}","{'size': 6300, 'units': 'sqft'}"
4,M1479304073,single_family,"{'city': 'San Diego', 'line': '15066 Almond Or...",1450000,1.0,5,5,"{'size': 5390, 'units': 'sqft'}","{'size': 23958, 'units': 'sqft'}"
...,...,...,...,...,...,...,...,...,...
195,M1267908716,single_family,"{'city': 'San Diego', 'line': '4338 E Overlook...",650000,0.0,2,3,"{'size': 1092, 'units': 'sqft'}","{'size': 5500, 'units': 'sqft'}"
196,M2826466121,single_family,"{'city': 'San Diego', 'line': '4175 Portilla P...",1250000,0.0,2,3,"{'size': 2008, 'units': 'sqft'}","{'size': 8949, 'units': 'sqft'}"
197,M1664428099,single_family,"{'city': 'San Diego', 'line': '5957 Del Cerro ...",804999,0.0,3,3,"{'size': 2109, 'units': 'sqft'}","{'size': 28314, 'units': 'sqft'}"
198,M2262370174,single_family,"{'city': 'San Diego', 'line': '4971 Cimarron W...",642000,0.0,3,5,"{'size': 2333, 'units': 'sqft'}","{'size': 4464, 'units': 'sqft'}"


In [8]:
from pandas.io.json import json_normalize
sandiego_address = json_normalize(sandiego_listings['address'])
sandiego_address

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,city,line,postal_code,state_code,state,county,fips_code,lat,lon,neighborhood_name,time_zone
0,San Diego,3910 Mount Blackburn Ave,92111,CA,California,San Diego,06073,32.812137,-117.186847,Northern San Diego,NaN
1,San Diego,8546 Boothbay Pl,92129,CA,California,San Diego,06073,32.951337,-117.140724,Penasquitos Park View Estates,NaN
2,La Jolla,1321 Virginia Way,92037,CA,California,San Diego,06073,32.844193,-117.268486,Northern San Diego,NaN
3,San Diego,4127 Seri St,92117,CA,California,San Diego,06073,32.821927,-117.202525,North Clairemont,NaN
4,San Diego,15066 Almond Orchard Ln,92131,CA,California,San Diego,06073,32.922956,-117.021887,Rancho Encantada,NaN
...,...,...,...,...,...,...,...,...,...,...,...
195,San Diego,4338 E Overlook Dr,92115,CA,California,San Diego,06073,32.755496,-117.070418,Mid-City,NaN
196,San Diego,4175 Portilla Pl,92130,CA,California,San Diego,06073,32.942724,-117.225369,Carmel Valley,NaN
197,San Diego,5957 Del Cerro Blvd,92120,CA,California,San Diego,06073,32.783419,-117.071399,Eastern San Diego,NaN
198,San Diego,4971 Cimarron Way,92154,CA,California,San Diego,06073,32.565702,-117.032011,Southern San Diego,NaN


In [9]:
sandiego_df = pd.concat([sandiego_listings,sandiego_address],axis=1)
sandiego_df.dropna(subset=['lot_size','building_size'], inplace=True)
sandiego_df.reset_index(inplace=True)
sandiego_df

,index,property_id,prop_type,address,price,baths_half,baths_full,beds,building_size,lot_size,city,line,postal_code,state_code,state,county,fips_code,lat,lon,neighborhood_name,time_zone
0,0,M2278071853,single_family,"{'city': 'San Diego', 'line': '3910 Mount Blac...",829000,0.0,2,3,"{'size': 1352, 'units': 'sqft'}","{'size': 6500, 'units': 'sqft'}",San Diego,3910 Mount Blackburn Ave,92111,CA,California,San Diego,06073,32.812137,-117.186847,Northern San Diego,NaN
1,1,M2739002954,single_family,"{'city': 'San Diego', 'line': '8546 Boothbay P...",699000,0.0,2,3,"{'size': 1614, 'units': 'sqft'}","{'size': 5718, 'units': 'sqft'}",San Diego,8546 Boothbay Pl,92129,CA,California,San Diego,06073,32.951337,-117.140724,Penasquitos Park View Estates,NaN
2,2,M1281042411,single_family,"{'city': 'La Jolla', 'line': '1321 Virginia Wa...",2395000,1.0,2,4,"{'size': 2586, 'units': 'sqft'}","{'size': 7700, 'units': 'sqft'}",La Jolla,1321 Virginia Way,92037,CA,California,San Diego,06073,32.844193,-117.268486,Northern San Diego,NaN
3,3,M1811103700,single_family,"{'city': 'San Diego', 'line': '4127 Seri St', ...",699999,0.0,2,3,"{'size': 1374, 'units': 'sqft'}","{'size': 6300, 'units': 'sqft'}",San Diego,4127 Seri St,92117,CA,California,San Diego,06073,32.821927,-117.202525,North Clairemont,NaN
4,4,M1479304073,single_family,"{'city': 'San Diego', 'line': '15066 Almond Or...",1450000,1.0,5,5,"{'size': 5390, 'units': 'sqft'}","{'size': 23958, 'units': 'sqft'}",San Diego,15066 Almond Orchard Ln,92131,CA,California,San Diego,06073,32.922956,-117.021887,Rancho Encantada,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,195,M1267908716,single_family,"{'city': 'San Diego', 'line': '4338 E Overlook...",650000,0.0,2,3,"{'size': 1092, 'units': 'sqft'}","{'size': 5500, 'units': 'sqft'}",San Diego,4338 E Overlook Dr,92115,CA,California,San Diego,06073,32.755496,-117.070418,Mid-City,NaN
188,196,M2826466121,single_family,"{'city': 'San Diego', 'line': '4175 Portilla P...",1250000,0.0,2,3,"{'size': 2008, 'units': 'sqft'}","{'size': 8949, 'units': 'sqft'}",San Diego,4175 Portilla Pl,92130,CA,California,San Diego,06073,32.942724,-117.225369,Carmel Valley,NaN
189,197,M1664428099,single_family,"{'city': 'San Diego', 'line': '5957 Del Cerro ...",804999,0.0,3,3,"{'size': 2109, 'units': 'sqft'}","{'size': 28314, 'units': 'sqft'}",San Diego,5957 Del Cerro Blvd,92120,CA,California,San Diego,06073,32.783419,-117.071399,Eastern San Diego,NaN
190,198,M2262370174,single_family,"{'city': 'San Diego', 'line': '4971 Cimarron W...",642000,0.0,3,5,"{'size': 2333, 'units': 'sqft'}","{'size': 4464, 'units': 'sqft'}",San Diego,4971 Cimarron Way,92154,CA,California,San Diego,06073,32.565702,-117.032011,Southern San Diego,NaN


In [10]:
building_size = json_normalize(sandiego_df['building_size'])
lot_size = json_normalize(sandiego_df['lot_size'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


In [14]:
sandiego_single_fam_df = pd.concat([sandiego_listings, sandiego_address, building_size],axis=1)
sandiego_single_fam_df

,property_id,prop_type,address,price,baths_half,baths_full,beds,building_size,lot_size,city,line,postal_code,state_code,state,county,fips_code,lat,lon,neighborhood_name,time_zone,size,units
0,M2278071853,single_family,"{'city': 'San Diego', 'line': '3910 Mount Blac...",829000,0.0,2,3,"{'size': 1352, 'units': 'sqft'}","{'size': 6500, 'units': 'sqft'}",San Diego,3910 Mount Blackburn Ave,92111,CA,California,San Diego,06073,32.812137,-117.186847,Northern San Diego,NaN,1352.0,sqft
1,M2739002954,single_family,"{'city': 'San Diego', 'line': '8546 Boothbay P...",699000,0.0,2,3,"{'size': 1614, 'units': 'sqft'}","{'size': 5718, 'units': 'sqft'}",San Diego,8546 Boothbay Pl,92129,CA,California,San Diego,06073,32.951337,-117.140724,Penasquitos Park View Estates,NaN,1614.0,sqft
2,M1281042411,single_family,"{'city': 'La Jolla', 'line': '1321 Virginia Wa...",2395000,1.0,2,4,"{'size': 2586, 'units': 'sqft'}","{'size': 7700, 'units': 'sqft'}",La Jolla,1321 Virginia Way,92037,CA,California,San Diego,06073,32.844193,-117.268486,Northern San Diego,NaN,2586.0,sqft
3,M1811103700,single_family,"{'city': 'San Diego', 'line': '4127 Seri St', ...",699999,0.0,2,3,"{'size': 1374, 'units': 'sqft'}","{'size': 6300, 'units': 'sqft'}",San Diego,4127 Seri St,92117,CA,California,San Diego,06073,32.821927,-117.202525,North Clairemont,NaN,1374.0,sqft
4,M1479304073,single_family,"{'city': 'San Diego', 'line': '15066 Almond Or...",1450000,1.0,5,5,"{'size': 5390, 'units': 'sqft'}","{'size': 23958, 'units': 'sqft'}",San Diego,15066 Almond Orchard Ln,92131,CA,California,San Diego,06073,32.922956,-117.021887,Rancho Encantada,NaN,5390.0,sqft
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,M1267908716,single_family,"{'city': 'San Diego', 'line': '4338 E Overlook...",650000,0.0,2,3,"{'size': 1092, 'units': 'sqft'}","{'size': 5500, 'units': 'sqft'}",San Diego,4338 E Overlook Dr,92115,CA,California,San Diego,06073,32.755496,-117.070418,Mid-City,NaN,NaN,NaN
196,M2826466121,single_family,"{'city': 'San Diego', 'line': '4175 Portilla P...",1250000,0.0,2,3,"{'size': 2008, 'units': 'sqft'}","{'size': 8949, 'units': 'sqft'}",San Diego,4175 Portilla Pl,92130,CA,California,San Diego,06073,32.942724,-117.225369,Carmel Valley,NaN,NaN,NaN
197,M1664428099,single_family,"{'city': 'San Diego', 'line': '5957 Del Cerro ...",804999,0.0,3,3,"{'size': 2109, 'units': 'sqft'}","{'size': 28314, 'units': 'sqft'}",San Diego,5957 Del Cerro Blvd,92120,CA,California,San Diego,06073,32.783419,-117.071399,Eastern San Diego,NaN,NaN,NaN
198,M2262370174,single_family,"{'city': 'San Diego', 'line': '4971 Cimarron W...",642000,0.0,3,5,"{'size': 2333, 'units': 'sqft'}","{'size': 4464, 'units': 'sqft'}",San Diego,4971 Cimarron Way,92154,CA,California,San Diego,06073,32.565702,-117.032011,Southern San Diego,NaN,NaN,NaN


In [0]:
sandiego_single_fam_df.drop(['address', 'building_size', 'state', 'fips_code', 'lat', 'lon', 'neighborhood_name', 'time_zone', 'units'], axis=1, inplace=True)

In [0]:
sandiego_single_fam_df.rename(columns={'size': "building_size"}, inplace=True)

In [20]:
sandiego_single_fam_df = pd.concat([sandiego_single_fam_df, lot_size],axis=1)
sandiego_single_fam_df

,property_id,prop_type,price,baths_half,baths_full,beds,lot_size,city,line,postal_code,state_code,county,building_size,size,units
0,M2278071853,single_family,829000,0.0,2,3,"{'size': 6500, 'units': 'sqft'}",San Diego,3910 Mount Blackburn Ave,92111,CA,San Diego,1352.0,6500.0,sqft
1,M2739002954,single_family,699000,0.0,2,3,"{'size': 5718, 'units': 'sqft'}",San Diego,8546 Boothbay Pl,92129,CA,San Diego,1614.0,5718.0,sqft
2,M1281042411,single_family,2395000,1.0,2,4,"{'size': 7700, 'units': 'sqft'}",La Jolla,1321 Virginia Way,92037,CA,San Diego,2586.0,7700.0,sqft
3,M1811103700,single_family,699999,0.0,2,3,"{'size': 6300, 'units': 'sqft'}",San Diego,4127 Seri St,92117,CA,San Diego,1374.0,6300.0,sqft
4,M1479304073,single_family,1450000,1.0,5,5,"{'size': 23958, 'units': 'sqft'}",San Diego,15066 Almond Orchard Ln,92131,CA,San Diego,5390.0,23958.0,sqft
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,M1267908716,single_family,650000,0.0,2,3,"{'size': 5500, 'units': 'sqft'}",San Diego,4338 E Overlook Dr,92115,CA,San Diego,NaN,NaN,NaN
196,M2826466121,single_family,1250000,0.0,2,3,"{'size': 8949, 'units': 'sqft'}",San Diego,4175 Portilla Pl,92130,CA,San Diego,NaN,NaN,NaN
197,M1664428099,single_family,804999,0.0,3,3,"{'size': 28314, 'units': 'sqft'}",San Diego,5957 Del Cerro Blvd,92120,CA,San Diego,NaN,NaN,NaN
198,M2262370174,single_family,642000,0.0,3,5,"{'size': 4464, 'units': 'sqft'}",San Diego,4971 Cimarron Way,92154,CA,San Diego,NaN,NaN,NaN


In [21]:
sandiego_single_fam_df.drop(['lot_size', 'units'], axis=1, inplace=True)
sandiego_single_fam_df.rename(columns={'size': "lot_size"}, inplace=True)
sandiego_single_fam_df.dropna(inplace=True)
sandiego_single_fam_df

,property_id,prop_type,price,baths_half,baths_full,beds,city,line,postal_code,state_code,county,building_size,lot_size
0,M2278071853,single_family,829000,0.0,2,3,San Diego,3910 Mount Blackburn Ave,92111,CA,San Diego,1352.0,6500.0
1,M2739002954,single_family,699000,0.0,2,3,San Diego,8546 Boothbay Pl,92129,CA,San Diego,1614.0,5718.0
2,M1281042411,single_family,2395000,1.0,2,4,La Jolla,1321 Virginia Way,92037,CA,San Diego,2586.0,7700.0
3,M1811103700,single_family,699999,0.0,2,3,San Diego,4127 Seri St,92117,CA,San Diego,1374.0,6300.0
4,M1479304073,single_family,1450000,1.0,5,5,San Diego,15066 Almond Orchard Ln,92131,CA,San Diego,5390.0,23958.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,M2046489914,single_family,1850000,1.0,2,3,San Diego,3965 Bandini St,92103,CA,San Diego,1092.0,5500.0
188,M1701272774,single_family,699000,1.0,2,4,San Diego,11099 Ice Skate Pl,92126,CA,San Diego,2008.0,8949.0
189,M1170579829,single_family,619000,0.0,2,3,San Diego,4219 Marlborough Ave,92105,CA,San Diego,2109.0,28314.0
190,M1691902911,single_family,1449000,1.0,5,5,San Diego,11419 Stockwood Cv,92131,CA,San Diego,2333.0,4464.0


In [22]:
sandiego_single_fam_final = pd.DataFrame(sandiego_single_fam_df)
sandiego_single_fam_final

,property_id,prop_type,price,baths_half,baths_full,beds,city,line,postal_code,state_code,county,building_size,lot_size
0,M2278071853,single_family,829000,0.0,2,3,San Diego,3910 Mount Blackburn Ave,92111,CA,San Diego,1352.0,6500.0
1,M2739002954,single_family,699000,0.0,2,3,San Diego,8546 Boothbay Pl,92129,CA,San Diego,1614.0,5718.0
2,M1281042411,single_family,2395000,1.0,2,4,La Jolla,1321 Virginia Way,92037,CA,San Diego,2586.0,7700.0
3,M1811103700,single_family,699999,0.0,2,3,San Diego,4127 Seri St,92117,CA,San Diego,1374.0,6300.0
4,M1479304073,single_family,1450000,1.0,5,5,San Diego,15066 Almond Orchard Ln,92131,CA,San Diego,5390.0,23958.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,M2046489914,single_family,1850000,1.0,2,3,San Diego,3965 Bandini St,92103,CA,San Diego,1092.0,5500.0
188,M1701272774,single_family,699000,1.0,2,4,San Diego,11099 Ice Skate Pl,92126,CA,San Diego,2008.0,8949.0
189,M1170579829,single_family,619000,0.0,2,3,San Diego,4219 Marlborough Ave,92105,CA,San Diego,2109.0,28314.0
190,M1691902911,single_family,1449000,1.0,5,5,San Diego,11419 Stockwood Cv,92131,CA,San Diego,2333.0,4464.0


In [24]:
from google.colab import drive
drive.mount('/drive', force_remount=True)
anaheim_single_fam_final.to_csv('/drive/My Drive/UCI-IRV-2020-04/sandiego_single_family_listings.csv')

Mounted at /drive


NameError: ignored